In [1]:
import numpy as np
import pandas as pd

import string
from scipy.misc import logsumexp
from sklearn import cluster
from sklearn.base import BaseEstimator, _pprint
from sklearn.utils import check_array, check_random_state
from sklearn.utils.validation import check_is_fitted

from sklearn.mixture import(distribute_covar_matrix_to_match_covariance_type, _validate_covars)
from sklearn.utils import check_random_state

In [2]:
def get_seq_serial(fname):
    # Serial Implementation

    df = pd.read_csv(fname)
    seq = list(df['Data'])
    seq = np.array(seq)

    return seq

def log_mask_zero(a):
    
    a = np.asarray(a)
    with np.errstate(divide="ignore"):
        return np.log(a)

def normalize(a, axis=None):
    """Normalizes the input array so that it sums to 1.

    Parameters
    ----------
    a : array
        Non-normalized input data.

    axis : int
        Dimension along which normalization is performed.

    Notes
    -----
    Modifies the input **inplace**.
    """
    a_sum = a.sum(axis)
    if axis and a.ndim > 1:
        # Make sure we don't divide by zero.
        a_sum[a_sum == 0] = 1
        shape = list(a.shape)
        shape[axis] = 1
        a_sum.shape = shape

    a /= a_sum
    

In [42]:
fname = 'no_blink.csv'

sequences = np.atleast_2d(get_seq_serial(fname))

X_input = sequences.T

In [64]:
i = 10

X = X_input[i:i+5]
# X = check_array(X)

X = np.hstack([X,X_input[i+5:i+10]]) # 2 features

X = check_array(X)

X 

# X.shape

# type(X)

array([[ 0.614286,  0.528571],
       [ 0.457143,  0.528571],
       [ 0.6     ,  0.728571],
       [ 0.5     ,  0.657143],
       [ 0.757143,  0.842857]])

In [5]:
## _BaseHMM class initialization parameters

n_components=2
covariance_type="diag"
n_iter=1000
startprob_prior = 1.0
transmat_prior=1.0
random_state=None
tol=1e-2 
verbose=False

In [6]:
## GMMHMM class initialization parameters

covariance_type = "diag"
min_covar = 1e-3
n_mix = 2
weights_prior = 1.0
means_prior = 0.0
means_weight = 0.0
covars_prior = None
covars_weight = None

In [7]:
# init

init = 1. / n_components

init

0.5

In [8]:
startprob_ = np.full(n_components, init)   # pi, initial probability

startprob_

array([ 0.5,  0.5])

In [9]:
transmat_ = np.full((n_components, n_components),init)  # Transition matrix (A) or Hidden Layer

transmat_

array([[ 0.5,  0.5],
       [ 0.5,  0.5]])

In [21]:
n_samples, n_features = X.shape

## Calling _init_covar_priors() function , covariance_type = "diag"
covars_prior = -1.5  
covars_weight = 0.0

In [11]:
# weights_prior : array, shape (n_mix, ), optional
#               Parameters of the Dirichlet prior distribution for
#               :attr:`weights_`.

weights_prior = np.broadcast_to(weights_prior, (n_components, n_mix)).copy()

weights_prior

array([[ 1.,  1.],
       [ 1.,  1.]])

In [76]:
# means_prior, means_weight : array, shape (n_mix, ), optional
#         Mean and precision of the Normal prior distribtion for
#         :attr:`means_`.

means_prior = np.broadcast_to(means_prior,(n_components, n_mix, n_features)).copy()

print "means_prior : "
print means_prior

means_weight = np.broadcast_to(means_weight,(n_components, n_mix)).copy()   # C matrix / Weight matrix

print "means_weight : "
print means_weight

means_ = np.zeros((n_components, n_mix,n_features))

print "means_ : "
print means_

means_prior : 
[[[ 0.  0.]
  [ 0.  0.]]

 [[ 0.  0.]
  [ 0.  0.]]]
means_weight : 
[[ 0.  0.]
 [ 0.  0.]]
means_ : 
[[[ 0.  0.]
  [ 0.  0.]]

 [[ 0.  0.]
  [ 0.  0.]]]


In [77]:
covars_prior = np.broadcast_to(covars_prior,(n_components, n_mix, n_features)).copy()

print "covars_prior"
print covars_prior

covars_weight = np.broadcast_to(covars_weight,(n_components, n_mix, n_features)).copy()

print "covars_weight"
print covars_weight

covars_prior
[[[-1.5 -1.5]
  [-1.5 -1.5]]

 [[-1.5 -1.5]
  [-1.5 -1.5]]]
covars_weight
[[[ 0.  0.]
  [ 0.  0.]]

 [[ 0.  0.]
  [ 0.  0.]]]


In [66]:
## Kmeans

main_kmeans = cluster.KMeans(n_clusters=n_components, random_state=random_state)
labels = main_kmeans.fit_predict(X)

labels

array([0, 0, 1, 0, 1], dtype=int32)

In [71]:
kmeanses = []

for label in range(n_components):
    kmeans = cluster.KMeans(n_clusters = n_mix,random_state = random_state)
    kmeans.fit(X[np.where(labels == label)])
    kmeanses.append(kmeans)

In [72]:
weights_ = (np.ones((n_components, n_mix)) /(np.ones((n_components, 1)) * n_mix))

weights_

array([[ 0.5,  0.5],
       [ 0.5,  0.5]])

In [78]:
for i, kmeans in enumerate(kmeanses):
    means_[i] = kmeans.cluster_centers_

means_

array([[[ 0.4785715,  0.592857 ],
        [ 0.614286 ,  0.528571 ]],

       [[ 0.757143 ,  0.842857 ],
        [ 0.6      ,  0.728571 ]]])

In [79]:
cv = np.cov(X.T) + min_covar*np.eye(n_features)
if not cv.shape:
    cv.shape = (1, 1)
        
covars_ = np.zeros((n_components, n_mix,n_features))
covars_[:] = np.diag(cv)

covars_

array([[[ 0.01457144,  0.01916329],
        [ 0.01457144,  0.01916329]],

       [[ 0.01457144,  0.01916329],
        [ 0.01457144,  0.01916329]]])

## Univariate Gaussian

### $P(X;\mu,\sigma^{2})$ = $ \frac{1}{\sqrt{2\pi}\sigma}\exp\Bigl(-\frac{1}{2\sigma^{2}}(x-\mu)^{2}\Bigr)$

In [80]:
def _log_multivariate_normal_density_diag(X, means, covars):
    """Compute Gaussian log-density at X for a diagonal model."""
    n_samples, n_dim = X.shape
    lpr = -0.5 * (n_dim * np.log(2 * np.pi) + np.sum(np.log(covars), 1)
                  + np.sum((means ** 2) / covars, 1)
                  - 2 * np.dot(X, (means / covars).T)
                  + np.dot(X ** 2, (1.0 / covars).T))
    return lpr

## Gaussian Mixture Model :

### $P(X;\mu,\sigma^{2})$ = $w_{1}*P(X_{1};\mu_{1},\sigma_{1}^{2})$ + $w_{2}*P(X_{2};\mu_{2},\sigma_{2}^{2})$
### $\sum_{m=1}^M w_{k} = 1$

In [81]:
def _compute_log_weighted_gaussian_densities(X, i_comp):
    cur_means = means_[i_comp]
    cur_covs = covars_[i_comp]
    log_cur_weights = np.log(weights_[i_comp])

    return _log_multivariate_normal_density_diag(X, cur_means, cur_covs) + log_cur_weights

In [89]:
def _compute_log_likelihood(X):
    n_samples, _ = X.shape
    res = np.zeros((n_samples, n_components))
    for i in range(n_components):
        log_denses = _compute_log_weighted_gaussian_densities(X, i)
#         print log_denses.T
        res[:, i] = logsumexp(log_denses, axis=1)
    return res

## Forward Propogation

In [90]:
framelogprob = _compute_log_likelihood(X)  # B matrix, or output probabilities 

framelogprob.T

array([[ 1.95084532,  1.83248274,  1.2357735 ,  1.82209547, -1.40781709],
       [ 0.61247574, -0.16362164,  1.82674396,  1.14996083,  1.82674396]])

In [91]:
log_startprob = log_mask_zero(startprob_)
log_transmat = log_mask_zero(transmat_)

n_samples, n_components = framelogprob.shape  # length * 2 pr 5 * 2

fwdlattice = np.zeros((n_samples, n_components))  # alpha

work_buffer = np.zeros(n_components)

### Initialization 
### $\alpha_{1}(i)$  = $\pi_{i}b_{i}(O_{1})$   1<=i<=N

In [92]:
for i in range(n_components):
    fwdlattice[0, i] = log_startprob[i] + framelogprob[0, i]
    
fwdlattice.T

array([[ 1.25769814,  0.        ,  0.        ,  0.        ,  0.        ],
       [-0.08067144,  0.        ,  0.        ,  0.        ,  0.        ]])

### Induction

### $\alpha_{t+1}(j)$  = $\Bigl[\sum_{i=1}^N\alpha_{t}(i)a_{ij}\Bigr]b_{j}O_{(t+1)}$   
1<=t<=T-1  <br> 1<=j<=N

In [93]:
for t in range(1, n_samples):
    for j in range(n_components):
        for i in range(n_components):
            work_buffer[i] = fwdlattice[t - 1, i] + log_transmat[i, j]
        fwdlattice[t, j] = logsumexp(work_buffer) + framelogprob[t, j]
        
print fwdlattice.T

# print np.exp(fwdlattice).T

[[ 1.25769814  2.62994771  3.29996721  5.4605828   3.77213699]
 [-0.08067144  0.63384333  3.89093767  4.78844817  7.00669804]]


### Termination : Forward Propogation
### $P(O|\lambda)$ = $\sum_{i=1}^N\alpha_{T}(i)$


## Backward Propogation


In [94]:
n_samples, n_components = framelogprob.shape

bwdlattice = np.zeros((n_samples, n_components))

work_buffer = np.zeros(n_components)

### Initialization :

### $\beta_{T}(i) = 1 $   

$1<=i<=N$


In [95]:
for i in range(n_components):
    bwdlattice[n_samples - 1, i] = 0.0    ## Log(1) = 0
    
bwdlattice.T

array([[ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.]])

### Induction
### $\beta_{t}(i)$ = $\sum_{j=1}^N a_{ij}b_{j}(O_{t+1})\beta_{t+1}(j)$

In [96]:
for t in range(n_samples - 2, -1, -1):
    for i in range(n_components):
        for j in range(n_components):
            work_buffer[j] = (log_transmat[i, j]
                              + framelogprob[t + 1, j]
                              + bwdlattice[t + 1, j])
        bwdlattice[t, i] = logsumexp(work_buffer)
        
bwdlattice.T

array([[ 5.55470785,  4.28797911,  2.71368549,  1.17221874,  0.        ],
       [ 5.55470785,  4.28797911,  2.71368549,  1.17221874,  0.        ]])

## $Posteriors$ : $\alpha(t)*\beta(t)$

In [97]:
# _compute_posteriors

log_gamma = fwdlattice + bwdlattice
log_normalize(log_gamma, axis=1)  
posteriors = np.exp(log_gamma)  

posteriors          

array([[ 0.79222169,  0.20777831],
       [ 0.88038746,  0.11961254],
       [ 0.35641222,  0.64358778],
       [ 0.66198097,  0.33801903],
       [ 0.03788564,  0.96211436]])

In [98]:
stats = {'nobs': 0,
         'start': np.zeros(n_components),
         'trans': np.zeros((n_components, n_components))}

stats['post'] = np.zeros(n_components)
stats['obs']  = np.zeros((n_components, n_features))
stats['obs**2'] = np.zeros((n_components, n_features))

## $\gamma_{t}(i)$ = $\frac{ \alpha_{t}(i)\beta_{t}(i)}{\sum_{i=1}^N \alpha_{t}(i)\beta_{t}(i)}$

## $\pi_{i} = \gamma_{1}(i) $

In [99]:
stats['start'] = posteriors[0]

n_samples, n_components = framelogprob.shape
log_xi_sum = np.full((n_components, n_components), -np.inf)
prob_mix = np.zeros((n_samples, n_components, n_mix))

stats['n_samples'] = n_samples  # 5
stats['samples'] = X

In [100]:
# Real output, B->O per weight value. Not logarithmic value Dimensions T*N*M

for p in range(n_components):
    log_denses = _compute_log_weighted_gaussian_densities(X, p)   
    prob_mix[:,p,:] = np.exp(log_denses) + np.finfo(np.float).eps  
    
prob_mix

array([[[ 2.2724724 ,  4.76215917],
        [ 0.17964221,  1.66535126]],

       [[ 4.2085438 ,  2.04083923],
        [ 0.01649425,  0.83256896]],

       [[ 1.77568789,  1.66535126],
        [ 1.4514627 ,  4.76215917]],

       [[ 4.2085438 ,  1.97626112],
        [ 0.2002683 ,  2.95780092]],

       [[ 0.06503459,  0.17964221],
        [ 4.76215917,  1.4514627 ]]])

In [101]:
prob_mix_sum = np.sum(prob_mix, axis=2)   # Dimensions T*N

prob_mix_sum

array([[ 7.03463156,  1.84499347],
       [ 6.24938303,  0.84906321],
       [ 3.44103915,  6.21362187],
       [ 6.18480492,  3.15806922],
       [ 0.24467681,  6.21362187]])

In [102]:
## Normalizing prob_mix

post_mix = prob_mix / prob_mix_sum[:, :, np.newaxis]
post_mix

array([[[ 0.32304071,  0.67695929],
        [ 0.0973674 ,  0.9026326 ]],

       [[ 0.67343349,  0.32656651],
        [ 0.01942641,  0.98057359]],

       [[ 0.51603246,  0.48396754],
        [ 0.23359366,  0.76640634]],

       [[ 0.68046508,  0.31953492],
        [ 0.06341479,  0.93658521]],

       [[ 0.26579795,  0.73420205],
        [ 0.76640634,  0.23359366]]])